# 1.1 Blocks

Bitcoin is commonly thought about as a ledger of transactions. Let's roll with that.

### Headers

We group the transactions into '**blocks**'. Each block has a **header**, which consists of the following attributes:

- Version: The version number in use.
- Previous hash: Each block has an identifier or hash. Here the 32-byte hash from the previous block is an attribute of the header of the current block.
- Timestamp: Unix timestamp in seconds.
- Nonce: A number that makes the hash of the header start with zeroes.

In [1]:
Hash = bytes

VERSION: int = 0

def init_header(previous_hash: Hash, timestamp: int, nonce: int) -> bytes:
    """ """
    return (
        VERSION.to_bytes(1, byteorder="big")
        + previous_hash
        + timestamp.to_bytes(4, byteorder="big")
        + nonce.to_bytes(32, byteorder="big")
    )

We use 'hash' as both a verb (to hash something) and a noun (the output of a hash function). It's a little confusing. To help clear things up, let's do something concrete - we solve for the nonce and create our first block.

Since there is no previous block, the previous hash is set to zero. We choose an arbitrary timestamp and initialize the nonce to zero. The header now looks like this.

In [2]:
previous_hash = (0).to_bytes(32, byteorder="big")
timestamp = 1634700000
nonce = 0

header = init_header(previous_hash, timestamp, nonce)
print(f"header:\n{header}")

header:
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00ao\x8a\xe0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'


We can make the header more readable by limiting each line to 16 bytes and separating each byte with a dash delimiter.

In [3]:
import binascii

def prettify(b: bytes) -> str:
    """ """
    n = len(b)
    m = (n // 16) + (n % 16 > 0)

    result = [
        binascii.hexlify(b[i * 16 : (i + 1) * 16], sep="-").decode()
        for i in range(m)
    ]
    return "\n".join(result)

print(f"header:\n{prettify(header)}")

header:
00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-61-6f-8a-e0-00-00-00-00-00-00-00-00-00-00-00
00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-00-00-00-00


OK let's hash. We use SHA-256 and hash the header twice. Even though only the header is hashed, we call it the block hash as it's used to identify the whole block.

In [4]:
import hashlib

guess = hashlib.sha256(hashlib.sha256(header).digest())
print(f"block_hash:\n{prettify(guess.digest())}")

block_hash:
48-9d-bd-f5-fd-58-0f-dc-0f-c3-72-54-6a-83-46-7e
10-01-71-84-09-f0-0c-96-25-90-72-7f-ba-21-cf-2a


Let's require the first two bytes be zero. Since each byte is represented by two hex digits, we need the first four characters in the no-delimiter string be zero.

In [5]:
print(f"block_hash - no delimiter:\n{guess.hexdigest()}\n")

is_new_block = guess.hexdigest()[:4] == "0000"
print(f"is_new_block: {is_new_block}")

block_hash - no delimiter:
489dbdf5fd580fdc0fc372546a83467e1001718409f00c962590727fba21cf2a

is_new_block: False


Now we keep incrementing the nonce until the first two bytes are zero.

In [6]:
previous_hash = (0).to_bytes(32, byteorder="big")
timestamp = 1634700000
nonce = 0

while True:
    header = init_header(previous_hash, timestamp, nonce)
    guess = hashlib.sha256(hashlib.sha256(header).digest())

    if guess.hexdigest()[:4] == "0000":
        break

    nonce += 1

print(f"nonce: {nonce}\n")
print(f"block_hash:\n{prettify(guess.digest())}\n")
print(f"header:\n{prettify(header)}")

nonce: 70822

block_hash:
00-00-1f-f5-84-95-c3-dc-2a-1a-aa-69-eb-ca-4e-9b
3e-05-e6-3e-5a-31-9f-b7-3b-cd-cc-bc-db-ba-1e-72

header:
00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-61-6f-8a-e0-00-00-00-00-00-00-00-00-00-00-00
00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-00-01-14-a6


Let's wrap the while loop in a function, and allow for a maximum number of iterations - this will be useful in later sections.

In [7]:
from typing import Optional, Tuple
import hashlib

def run_proof_of_work(
    previous_hash: Hash,
    timestamp: int,
    nonce: int = 0,
    iterations: Optional[int] = None,
) -> Tuple[bool, int, Optional[Hash], Optional[bytes]]:
    """Find nonce that makes the first 4 bytes of twice-hashed header all
    zeroes. Maximum number of iterations can be specified."""
    iteration_counter = 0

    while True:
        if iterations is not None and iteration_counter == iterations:
            return False, nonce, None, None

        header = init_header(previous_hash, timestamp, nonce)
        guess = hashlib.sha256(hashlib.sha256(header).digest())

        if guess.hexdigest()[:4] == "0000":
            break

        nonce += 1
        iteration_counter += 1

    return True, nonce, guess.digest(), header

### Transactions

For the time being, we restrict the transaction to the reward for mining the block. We use port numbers to signify each 'account'. Since there is no sender in a reward transaction, we set the sender to be zero. In other words, (i) the first transaction in each block is the reward for the miner, and (ii) the reward transaction is signified by having sender zero.

In [8]:
def init_transactions(node_port: int):
    """ """
    sender = (0).to_bytes(2, byteorder="big")
    receiver = node_port.to_bytes(2, byteorder="big")

    transaction_counter = 1
    transactions = sender + receiver

    return transaction_counter, transactions

We start with the first reward going to port 7000.

In [9]:
transaction_counter, transactions = init_transactions(7000)

print(f"transaction_counter: {transaction_counter}\n")
print(f"transactions:\n{prettify(transactions)}")

transaction_counter: 1

transactions:
00-00-1b-58


### Blocks

The block consists of the header and the transactions - we similarly wrap that in a function. The first byte of the block is reserved for the block size in bytes. The header size is fixed at 69 bytes i.e. 1 + 32 + 4 + 32.

In [10]:
HEADER_SIZE: int = 69

def init_block(header: bytes, transaction_counter: int, transactions: bytes) -> bytes:
    """ """
    block_size = 1 + HEADER_SIZE + 1 + len(transactions)

    return (
        block_size.to_bytes(1, byteorder="big")
        + header
        + transaction_counter.to_bytes(1, byteorder="big")
        + transactions
    )

Let's collect what what we have so far with a function that creates the very first block, commonly referred to as the **genesis block**.

In [11]:
def init_genesis_block() -> Tuple[Hash, bytes]:
    """ """
    previous_hash = (0).to_bytes(32, byteorder="big")
    timestamp = 1634700000
    nonce = 70822

    header = init_header(previous_hash, timestamp, nonce)
    guess = hashlib.sha256(hashlib.sha256(header).digest())
    assert guess.hexdigest()[:4] == "0000"

    transaction_counter, transactions = init_transactions(7000)
    block = init_block(header, transaction_counter, transactions)

    return guess.digest(), block

Finally we have in bytes (i) the hash of the the genesis block, and (ii) the genesis block.

In [12]:
genesis_hash, genesis_block = init_genesis_block()

print(f"genesis_hash:\n{prettify(genesis_hash)}\n")
print(f"genesis_block:\n{prettify(genesis_block)}")

genesis_hash:
00-00-1f-f5-84-95-c3-dc-2a-1a-aa-69-eb-ca-4e-9b
3e-05-e6-3e-5a-31-9f-b7-3b-cd-cc-bc-db-ba-1e-72

genesis_block:
4b-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-00-61-6f-8a-e0-00-00-00-00-00-00-00-00-00-00
00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-00-00-01-14-a6-01-00-00-1b-58


We've simplified a lot! Let's look at a number of things that different compared to the actual implementation.

- The mining difficulty is constant at two leading zero bytes. The actual implementation has a difficulty that changes so that a new block is mined, on average, every 10 minutes.
- The reward for mining a block is one coin. The actual implementation started at 50 coins in 2009, and halves roughly every 4 years.
- The account is represented by port numbers. The actual implementation uses public key cryptography to generate accounts.

In the next section, we'll be mining more blocks and 'chain' them together.

# 1.2 Chains

In the previous post, we introduced mining and created the genesis block. Let's mine more blocks and 'chain' them together - we call this the **blockchain**.

### Initializing the blockchain

The genesis block is set as the first block in the blockchain. We also keep a running counter how many blocks have been mined. 

In [13]:
genesis_hash, genesis_block = init_genesis_block()

blockchain = genesis_block
block_counter = 1

As before, we use port numbers to represent accounts.

In [14]:
port = 7000

Let's review steps 1 to 4 in mining a new block, introduced previously.

1. Get the hash of the previous block, and combine that with the timestamp and nonce to create a header.
2. Hash the header twice.
3. If the first two bytes are not zero, increment the nonce and try again.
4. If the first two bytes are zero, create a reward transaction and combine that with the header to create a new block.

In addition, we introduce steps 5 to 6. We run all the steps to mine up to 10 blocks.

1. Append the new block to the blockchain.
2. Re-initialize the previous hash, timestamp and nonce values to mine the next block.

For our purposes, the first non-zero previous hash is the hash of the genesis block. We set up the loop to iterate through 1,000 nonce values at a time - we'll see why in the next post. Let's also print out (without delimiters) the block hash every time a new block is mined.

In [15]:
previous_hash = genesis_hash
timestamp = 1634700600
nonce = 0

print(
    f"CREATE block {block_counter - 1}: {binascii.hexlify(genesis_hash).decode()}"
)

while True:
    is_new_block, _, current_hash, header = run_proof_of_work(
        previous_hash, timestamp, nonce, 1000
    )

    # Retry if not solved after 1000 nonce values.
    if not is_new_block:
        nonce += 1000
        continue

    # Create transaction to award block if solved.
    transaction_counter, transactions = init_transactions(port)
    block = init_block(header, transaction_counter, transactions)

    blockchain += block
    block_counter += 1

    print(
        f"CREATE block {block_counter - 1}: {binascii.hexlify(current_hash).decode()}"
    )

    if block_counter >= 10:
        break
    
    # Reset values for next block header.
    previous_hash = current_hash
    timestamp += 600
    nonce = 0

CREATE block 0: 00001ff58495c3dc2a1aaa69ebca4e9b3e05e63e5a319fb73bcdccbcdbba1e72
CREATE block 1: 000071e6ff5b358e57339b42d45b20acc0f112c218fa435b3ffa8f239b777347
CREATE block 2: 0000e7d62e199111cc9da5227d7029c8e1224a40a2927e312596732835947e7d
CREATE block 3: 00008d85f05207e0db65b0d5c15136ec45ee57a52b9189c8b85e8126435cf6d1
CREATE block 4: 000098989806054e1d3cf0b9a061e7b4492251300dc8d2281424612af7660280
CREATE block 5: 000035736dd26882f64b58d6a95c37ebf849dc8a6d3a4d55083e0c4498440976
CREATE block 6: 0000e136c7f750925dec1f4f6f9d806d1274c90249788042d335de32db69641f
CREATE block 7: 00000953bf124a4c47ea23175c0c0dd0a66bb0841f4428f205032fb7b1274f97
CREATE block 8: 000073bfe1dc76721bcf70e40d2ec5520d969c60170a8d6bc50b09c9a95526ba
CREATE block 9: 000074271e92a82bf58f083d536444f7a12dba90f6432d03de0d573cf0979e38


Each block is 75 bytes i.e. 1 + 69 + 1 + 4, so a blockchain with 10 blocks would be 750 bytes.

In [16]:
print(f"blockchain:\n{prettify(blockchain)}")

blockchain:
4b-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-00-61-6f-8a-e0-00-00-00-00-00-00-00-00-00-00
00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-00-00-01-14-a6-01-00-00-1b-58-4b-00-00-00-1f
f5-84-95-c3-dc-2a-1a-aa-69-eb-ca-4e-9b-3e-05-e6
3e-5a-31-9f-b7-3b-cd-cc-bc-db-ba-1e-72-61-6f-8d
38-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-e5
90-01-00-00-1b-58-4b-00-00-00-71-e6-ff-5b-35-8e
57-33-9b-42-d4-5b-20-ac-c0-f1-12-c2-18-fa-43-5b
3f-fa-8f-23-9b-77-73-47-61-6f-8f-90-00-00-00-00
00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-00-00-00-00-00-00-00-00-00-0f-17-01-00-00-1b
58-4b-00-00-00-e7-d6-2e-19-91-11-cc-9d-a5-22-7d
70-29-c8-e1-22-4a-40-a2-92-7e-31-25-96-73-28-35
94-7e-7d-61-6f-91-e8-00-00-00-00-00-00-00-00-00
00-00-00-00-00-00-00-00-00-00-00-00-00-00-00-00
00-00-00-00-01-8c-7a-01-00-00-1b-58-4b-00-00-00
8d-85-f0-52-07-e0-db-65-b0-d5-c1-51-36-ec-45-ee
57-a5-2b-91-89-c8-b8-5e-81-2

### Validating the blockchain

Suppose we have the opposite problem, in which we are given a blockchain and need to check that it is indeed valid. To make iterating through each block easier, let's create a helper function.

In [17]:
from typing import Generator

def iterate_blockchain(blockchain: bytes, byte_index: int = 0) -> Generator:
    """Helper function to simplify iterating through blocks in the blockchain."""
    block_counter = 0

    while True:
        block_size = blockchain[byte_index]
        block = blockchain[byte_index : byte_index + block_size]

        byte_index += block_size
        block_counter += 1

        yield block, block_counter, byte_index

For validation, we check that:

1. The previous hash attribute in the current header matches the hash of the previous block.
2. The hash of the current header starts with two zero bytes.

In [18]:
HASH_SIZE: int = 32

def validate_blockchain(blockchain: bytes) -> Tuple[bool, int, Optional[bytes]]:
    """Check that all headers in the blockchain satisfy proof-of-work and indeed form a chain."""
    previous_hash = (0).to_bytes(32, byteorder="big")
    blockchain_size = len(blockchain)

    for block, block_counter, byte_index in iterate_blockchain(
        blockchain, byte_index=0
    ):
        header = block[1 : 1 + HEADER_SIZE]

        if header[1 : 1 + HASH_SIZE] != previous_hash:
            return False, block_counter, None

        guess = hashlib.sha256(hashlib.sha256(header).digest())

        if guess.hexdigest()[:4] != "0000":
            return False, block_counter, None

        previous_hash = guess.digest()

        if byte_index == blockchain_size:
            break

    return True, block_counter, previous_hash

Now let's run our blockchain through the validation function, to confirm the result is as expected.

In [19]:
is_valid, block_counter, latest_hash = validate_blockchain(blockchain)

print(f"is_valid: {is_valid}\n")
print(f"block_counter: {block_counter}\n")
print(f"latest_hash:\n{prettify(latest_hash)}")

is_valid: True

block_counter: 10

latest_hash:
00-00-74-27-1e-92-a8-2b-f5-8f-08-3d-53-64-44-f7
a1-2d-ba-90-f6-43-2d-03-de-0d-57-3c-f0-97-9e-38


In the next section, we'll implement multiple mining nodes that broadcast to each other.

# 1.3 Nodes

In the previous post, we introduced a single miner. Now let's set up 3 nodes that mine and broadcast to each other.

### Context

Let's start by sketching out what each node would do. In mining mode, the node tries 1,000 nonce values at a time and extends the blockchain if a solution is found. In listening mode, the node listens to the network and replaces its blockchain if the broadcasted blockchain is valid and longer than the existing blockchain. To keep things simple, the node would alternate between the two modes instead of running both in parallel.

### Basic nodes

Since most of our focus so far has been on mining, let's switch gears briefly to set up nodes that broadcast to each other. To ensure our nodes are set up correctly, we create a `.env` file with the IP address by running the following command in a terminal window.

Next we load the IP address as an environment variable, and enumerate all the ports to be used.

In [20]:
import dotenv
import os

dotenv.load_dotenv()

NODE_IP = os.getenv("NODE_IP")
NODE_PORTS = [7000, 8000, 9000]

We introduce a helper function to bind the network socket to the IP address-port pair.

In [21]:
import socket

def bind_socket(ip_address: str, port: int) -> socket.socket:
    """ """
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.bind((ip_address, port))
    return sock

Let's wrap the socket in a class and create functions to (i) initialize the node, and (ii) broadcast to other nodes. The node class also stores the blockchain state - we'll use this in the next section.

In [22]:
import dataclasses

@dataclasses.dataclass
class Node:
    """ """

    port: int
    sock: socket.socket
    blockchain: bytes
    block_counter: int

def init_node(port: int, blockchain: bytes = b"", block_counter: int = 0) -> Node:
    """ """
    assert NODE_IP is not None
    sock = bind_socket(NODE_IP, port)

    return Node(
        port=port, sock=sock, blockchain=blockchain, block_counter=block_counter
    )

def broadcast(node: Node, message: bytes):
    """ """
    for node_port in NODE_PORTS:
        if node_port == node.port:
            continue

        node.sock.sendto(message, (NODE_IP, node_port))

In the placeholder version, we set up the nodes to broadcast a simple message to the network and then sleep for a random time.

In [23]:
def run(node: Node):
    """ """
    while True:
        try:
            node.sock.settimeout(1)
            message, _ = node.sock.recvfrom(9216)

            node_port = int.from_bytes(message[:2], byteorder="big")
            sleep_time = message[2]

            print(f"OTHER {node_port} sleep for {sleep_time} seconds...")

        except socket.timeout:
            sleep_time = random.randint(1, 9)
            message = node.port.to_bytes(2, byteorder="big") + sleep_time.to_bytes(
                1, byteorder="big"
            )

            broadcast(node, message)
            print(f"SELF {node.port} sleep for {sleep_time} seconds...")

            time.sleep(sleep_time)

We set up main to take in the port number as a command line argument.

That's a fair amount of setup! Now respectively run each line in different terminal windows. A sample run can be seen here.

### Mining nodes

Now that we've done the heavy-lifting of setting up nodes that broadcast to each other, we can incorporate mining into the nodes.

At the start of this post, we discussed alternating between mining mode and listening mode. The steps in mining mode is similar to the previous post; the switch from mining to listening mode happens after trying 1,000 nonce values. The steps in listening mode are as follows.

1. Listen to incoming messages from other nodes.
2. If message is received, check broadcasted blockchain in the message is valid and longer than the existing blockchain.
    1. If the broadcasted blockchain is valid and longer than the existing blockchain, replace the stored blockchain with the broadcasted blockchain.
    2. If the broadcasted blockchain is invalid or not longer than the existing blockchain, ignore the broadcasted blockchain.
3. If no message is received after listening for 1 second, switch to mining mode.

Since the blockchain is an array of bytes, we can broadcast it directly! We modify the run function as follows.

In [24]:
import binascii
import time

def run(node: Node):
    """ """
    previous_hash = genesis_hash
    timestamp = int(time.time())
    nonce = 0

    while True:
        try:
            # Listen for incoming messages, with maximum size set at OS X
            # maximum UDP package size of 9216 bytes.
            node.sock.settimeout(1)
            blockchain, _ = node.sock.recvfrom(9216)

            # Check blockchain received is valid.
            is_new_block, block_counter, current_hash = blocks.validate_blockchain(
                blockchain
            )

            # Skip if invalid or not longer than existing blockchain.
            if not is_new_block or block_counter <= node.block_counter:
                print("IGNORE blockchain...")
                continue

            # Replace if valid and longer than existing.
            node.blockchain = blockchain
            node.block_counter = block_counter

            assert current_hash is not None
            print(
                f"COPY block {node.block_counter - 1}: {binascii.hexlify(current_hash).decode()}!"
            )

            # Force sleep to randomize timestamp.
            sleep_time = (node.port + block_counter) % 3 + 1
            print(f"SLEEP for {sleep_time} seconds...")
            time.sleep(sleep_time)

        except socket.timeout:
            # Run proof-of-work.
            print(f"TRY up to {nonce}...")
            is_new_block, _, current_hash, header = blocks.run_proof_of_work(
                previous_hash, timestamp, nonce, 1000
            )

            # Switch to listening mode if not solved after 1000 nonce values.
            if not is_new_block:
                nonce += 1000
                continue

            # Create transaction to award block if solved.
            assert current_hash is not None and header is not None
            transaction_counter, transactions = blocks.init_transactions(node.port)
            block = blocks.init_block(header, transaction_counter, transactions)

            node.blockchain += block
            node.block_counter += 1

            # Broadcast full blockchain to network.
            broadcast(node, node.blockchain)
            print(
                f"CREATE block {node.block_counter - 1}: {binascii.hexlify(current_hash).decode()}!"
            )

        # Reset values for next block header.
        previous_hash = current_hash
        timestamp = int(time.time())
        nonce = 0

With the changes in place, we re-run each line below in different terminal windows.

In the next series of posts, we'll take a closer look at transactions and validating that each coin is only spent once.